In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import seaborn as sns
import plotly.plotly as py
import matplotlib.pyplot as plt
import cufflinks as cf 
from ipywidgets import interact
import plotly.graph_objs as go
from rpdb import read_table, export_table

In [ ]:
# function takes vertical speed(vs) and horitzontal speed (hs) as an arguments, and returns GAP
def gap_conversion(vs,hs):
    HEIGHT = 1.8
    WEIGHT = 77
    Af = (0.2025 * HEIGHT**0.725 * WEIGHT**0.425) * 0.266
    Kaero = 0.5 * 1.2 * 0.9 * Af   #0.5 * rho * Cd * Af with rho(air) = 1.2, Cd = 0.9
    if hs == 0.0:
        gap = 0.0
    else:
        # parameters
        slope = vs/hs
        Kslope = ((((155.4 * slope - 30.4) * slope - 43.3) * slope + 46.3) * slope + 19.5) * slope + 3.6
        Eff = (-0.0032413 * (hs**2 + vs**2)**0.5 + 0.038994) * (hs**2 + vs**2)**0.5 + 0.25
        
        # power calculation based on slope and speed
        Epot = WEIGHT * 9.8 * vs/1 
        Ecin = 0.5 * WEIGHT * (hs**2 + vs**2)
        Eaero = Kaero * (hs**2 + vs**2)
        E = Epot + Ecin + Eaero
        
        if -Epot > Ecin + Eaero: #The coefficients doesn't take into account high-downhill-rate, so we assume for these rows gap = hs
            E = Ecin + Eaero 
        
        # GAP adjustement: based on E = E_eq and convertion to pace (min / km)
        adj_speed = (E / (Kaero + 0.5 * WEIGHT))**0.5
        gap = round(1000 / (adj_speed * 60), 2)
    
    return gap

In [2]:
records = read_table('records')
records.sample(n=5)

,index,idmove,idrecord,latitude,longitude,distance,altitude,enhanced_altitude,cadence,speed,enhanced_speed,vertical_speed,heart_rate,timestamp,pace,accum_ascent
0,0,1,6,495381218.0,28361471.0,14.0,3165.0,133.0,90.0,2100.0,2.1,0.0,110.0,2017-07-02 09:46:48,7.94,0.0
1,1,1,7,495381528.0,28361399.0,17.0,3165.0,133.0,90.0,2100.0,2.1,0.0,110.0,2017-07-02 09:46:49,7.94,0.0
2,2,1,8,495381826.0,28361339.0,20.0,3165.0,133.0,90.0,2500.0,2.5,0.0,110.0,2017-07-02 09:46:50,6.67,0.0
3,3,1,9,495382113.0,28361268.0,22.0,3170.0,134.0,90.0,2500.0,2.5,0.0,110.0,2017-07-02 09:46:51,6.67,1.0
4,4,1,10,495382411.0,28361184.0,25.0,3170.0,134.0,90.0,2600.0,2.6,0.0,110.0,2017-07-02 09:46:52,6.41,1.0


In [13]:
records['gap'] = records.apply(lambda x: gap_conversion(x['vertical_speed'], x['enhanced_speed']), axis = 1)